# Answering Business Questions With SQL

This notebook will go over using the sqlite3 module and some pandas to query a sqlite database to answer some business questions about album sales, purchase behaviors, etc.

In [1]:
import pandas as pd
import sqlite3

## Helper Functions

Let's start off by making the following helper functions:

run_query(): takes a SQL query as an argument and returns a pandas dataframe of that query.

In [2]:
def run_query(query):
    with sqlite3.connect("chinook.db") as conn:
        return pd.read_sql(query, conn)

run_command(): takes a SQL command as an argument and executes it using the sqlite module.

In [3]:
def run_command(query):
    with sqlite3.connect("chinook.db") as conn:
        conn.isolation_level = None
        conn.execute(query)

show_tables(): calls the run_query() function to return a list of all tables and views in the database.

In [4]:
def show_tables():
    query = """
        SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ("table","view");
    """
    return run_query(query)

Now, let's run the show_tables() function.

In [5]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a record label. We are tasked with selecting 3 albums from a list of 4 albums to add to the store. The record label has given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We will start by writing a query that returns each genre and the number of tracks sold in the USA in both absolute numbers and percentages.

In [6]:
best_selling_usa_genres_query = """
    WITH tracks_sold_by_genre AS (
        SELECT
            g.name AS genre,
            COUNT(t.track_id) AS num_tracks_sold
        FROM
            track t
            LEFT JOIN genre g
                ON t.genre_id = g.genre_id
            LEFT JOIN invoice_line il
                ON t.track_id = il.track_id
            LEFT JOIN invoice i
                ON il.invoice_id = i.invoice_id
        WHERE
            i.billing_country = "USA"
        GROUP BY genre
    )
    SELECT
        genre,
        num_tracks_sold,
        ROUND(100.0 * num_tracks_sold / (SELECT SUM(num_tracks_sold) FROM tracks_sold_by_genre), 2) AS pct_of_usa_tracks_sold
    FROM tracks_sold_by_genre
    ORDER BY num_tracks_sold DESC;
"""

run_query(best_selling_usa_genres_query)

,genre,num_tracks_sold,pct_of_usa_tracks_sold
0,Rock,561,53.38
1,Alternative & Punk,130,12.37
2,Metal,124,11.80
3,R&B/Soul,53,5.04
4,Blues,36,3.43
5,Alternative,35,3.33
6,Latin,22,2.09
7,Pop,22,2.09
8,Hip Hop/Rap,20,1.90
9,Jazz,14,1.33


The 4 tracks that we have to choose from are:

| Artist Name | Genre |
| ----------- | ----------- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

Based on our table of genres and number of tracks sold in the USA, we should purchase Red Tone (Punk), Slim Jim Bites (Blues), Meteor and the Girls (Pop) albums for the store sinc

## Analyzing Sales By Country
Will will now analyze sales for each country. For each country, we will look at total number of customers, total value of sales, average value of sales per customer, and average order value. In addition, there will be a couple countries with only one customer. These countries will be grouped under "Other" country.

In [7]:
sales_by_country = '''
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );
'''

run_query(sales_by_country)

,country,customers,total_sales,average_order,customer_lifetime_value
0,USA,13,1040.49,7.942672,80.037692
1,Canada,8,535.59,7.047237,66.948750
2,Brazil,5,427.68,7.011148,85.536000
3,France,5,389.07,7.781400,77.814000
4,Germany,4,334.62,8.161463,83.655000
5,Czech Republic,2,273.24,9.108000,136.620000
6,United Kingdom,3,245.52,8.768571,81.840000
7,Portugal,2,185.13,6.383793,92.565000
8,India,2,183.15,8.721429,91.575000
9,Other,15,1094.94,7.448571,72.996000


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums Vs Individual Tracks

Currently, the Chinook record store allows customers to either purchase individual tracks or purchase an entire album in a single purchase. Customers are not allowed to do both. Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

There are two edge cases to consider:

* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

We will write a query that categorizes each invoice as an album purchase or individual track(s) purchase to determine how sound a strategy it would be if we stopped purchasing entire albums and only purchased the most popular tracks from each album.

In [8]:
album_vs_track_query = """
    WITH
    invoice_first_track_id AS (
        SELECT
            il.invoice_id AS invoice_id,
            MIN(track_id) AS first_track_id
        FROM invoice_line il
        GROUP BY invoice_id
    ),
    is_album_purchase AS (
        SELECT
            ifti.*,
            (
                SELECT track_id
                FROM invoice_line il
                WHERE invoice_id = ifti.invoice_id
                EXCEPT
                SELECT track_id
                FROM track t
                WHERE t.album_id = (
                    SELECT album_id 
                    FROM track
                    WHERE track_id = ifti.first_track_id
                )
            ) IS NULL AND
            (
                SELECT track_id
                FROM track t
                WHERE t.album_id = (
                    SELECT album_id 
                    FROM track
                    WHERE track_id = ifti.first_track_id
                )
                EXCEPT
                SELECT track_id
                FROM invoice_line il
                WHERE invoice_id = ifti.invoice_id                
                IS NULL
            ) AS is_album_purchase
        FROM invoice_first_track_id ifti
    )
    SELECT
        (SELECT COUNT(invoice_id) FROM is_album_purchase WHERE is_album_purchase = 1) AS num_album_purchases,
        (SELECT COUNT(invoice_id) FROM is_album_purchase WHERE is_album_purchase = 0) AS num_single_purchases,
        (SELECT 100.0 * COUNT(invoice_id) FROM is_album_purchase WHERE is_album_purchase = 1) / 
        (SELECT COUNT(invoice_id) FROM is_album_purchase) AS pct_album_purchases,
        (SELECT 100.0 * COUNT(invoice_id) FROM is_album_purchase WHERE is_album_purchase = 0) / 
        (SELECT COUNT(invoice_id) FROM is_album_purchase) AS pct_single_purchases
"""
        
    
run_query(album_vs_track_query)

,num_album_purchases,num_single_purchases,pct_album_purchases,pct_single_purchases
0,171,443,27.850163,72.149837


Based on the above findings, it looks like Chinook should still allow customers to purchase single tracks as opposed to just entire albums, as the large majority of invoices are not album purchases (%72.15)

## Additional Business Questions

Let's look into some other interesting business questions:

Which artist is used in the most playlists?

In [9]:
most_popular_artist_query = """
    SELECT
        art.artist_id AS artist_id,
        art.name AS artist_name,
        COUNT(DISTINCT pt.playlist_id) AS num_playlists
    FROM playlist_track pt
    LEFT JOIN track t
        ON pt.track_id = t.track_id
    LEFT JOIN album alb
        ON t.album_id = alb.album_id
    LEFT JOIN artist art
        ON alb.artist_id = art.artist_id
    GROUP BY art.artist_id, art.name
    ORDER BY num_playlists DESC
    LIMIT 10;
"""

run_query(most_popular_artist_query)

,artist_id,artist_name,num_playlists
0,226,Eugene Ormandy,7
1,208,English Concert & Trevor Pinnock,6
2,214,Academy of St. Martin in the Fields & Sir Nevi...,6
3,247,The King's Singers,6
4,248,Berliner Philharmoniker & Herbert Von Karajan,6
5,206,Alberto Turco & Nova Schola Gregoriana,5
6,207,"Richard Marlow & The Choir of Trinity College,...",5
7,211,Wilhelm Kempff,5
8,212,Yo-Yo Ma,5
9,213,Scholars Baroque Ensemble,5


Eugene Ormandy is used in the most playlists (7).

How many tracks have been purchased vs not purchased?

In [10]:
tracks_purchased_vs_not_purchased_query = """
    WITH 
    purchased_tracks AS (
        SELECT DISTINCT track_id FROM invoice_line
    ),
    unpurchased_tracks AS (
        SELECT DISTINCT track_id FROM track
        EXCEPT
        SELECT track_id FROM purchased_tracks
    )
    SELECT
        (SELECT COUNT(track_id) FROM purchased_tracks) AS num_purchased_tracks,
        (SELECT COUNT(track_id) FROM unpurchased_tracks) AS num_unpurchased_tracks;
"""

run_query(tracks_purchased_vs_not_purchased_query)

,num_purchased_tracks,num_unpurchased_tracks
0,1806,1697


Is the range of tracks in the store reflective of their sales popularity?

In [11]:
store_tracks_genres_query = """
    WITH store_tracks AS (
        SELECT
            g.name AS genre,
            COUNT(track_id) AS num_tracks
        FROM track t
        LEFT JOIN genre g
        ON t.genre_id = g.genre_id
        GROUP BY g.name
        ORDER BY num_tracks DESC
    ),
    tracks_sold_by_genre AS (
        SELECT
            g.name AS genre,
            COUNT(t.track_id) AS num_tracks_sold
        FROM
            track t
            LEFT JOIN genre g
                ON t.genre_id = g.genre_id
            LEFT JOIN invoice_line il
                ON t.track_id = il.track_id
            LEFT JOIN invoice i
                ON il.invoice_id = i.invoice_id
        GROUP BY genre
    )
    SELECT
        st.genre AS genre,
        st.num_tracks AS num_tracks,
        ROUND(100.0 * num_tracks / (SELECT SUM(num_tracks) FROM store_tracks), 2) AS pct_of_tracks,
        tsbg.num_tracks_sold AS num_tracks_sold,
        ROUND(100.0 * num_tracks_sold / (SELECT SUM(num_tracks_sold) FROM tracks_sold_by_genre), 2) AS pct_of_tracks_sold
    FROM store_tracks st
    LEFT JOIN tracks_sold_by_genre tsbg
    ON st.genre = tsbg.genre
    ORDER BY num_tracks_sold DESC;
"""

run_query(store_tracks_genres_query)

,genre,num_tracks,pct_of_tracks,num_tracks_sold,pct_of_tracks_sold
0,Rock,1297,37.03,3017,46.75
1,Metal,374,10.68,755,11.70
2,Alternative & Punk,332,9.48,648,10.04
3,Latin,579,16.53,627,9.71
4,Jazz,130,3.71,190,2.94
5,R&B/Soul,61,1.74,165,2.56
6,Blues,81,2.31,149,2.31
7,Alternative,40,1.14,123,1.91
8,Classical,74,2.11,105,1.63
9,TV Shows,93,2.65,93,1.44


It looks like the distribution of tracks the store offers is more or less in line with the customer purchasing behavior. The top 5 most purchased genres are Rock, Metal, Alternative & Punk, Latin, and Jazz. The top 5 most available genres in the store are Rock, Latin, Metal, Alternative & Punk, and Jazz. The top 5 are the same with the order of the genres slightly different (the store's 2nd most common genre is Latin, while Latin is the 4th most purchased genre).

Do protected vs non-protected media types have an effect on popularity?

In [12]:
protected_media_type_query = """
    WITH 
    protected_media_types AS (
        SELECT 
            media_type_id,
            CASE WHEN
                    name LIKE '%Protected%'
                THEN "Protected"
                ELSE "Unprotected"
            END 
            AS protected
        FROM media_type
    )
    SELECT
        pmt.protected AS protected,
        COUNT(DISTINCT il.track_id) AS num_tracks,
        100.0 * COUNT(DISTINCT t.track_id) / (SELECT COUNT(DISTINCT track_id) FROM invoice_line) AS pct_tracks
    FROM invoice_line il 
    LEFT JOIN track t
        ON t.track_id = il.track_id
    LEFT JOIN protected_media_types pmt
        ON t.media_type_id = pmt.media_type_id
    GROUP BY protected;
"""

run_query(protected_media_type_query)

,protected,num_tracks,pct_tracks
0,Protected,154,8.527132
1,Unprotected,1652,91.472868


Protected vs non-protected media types definitely have an effect on popularity. Unprotected tracks make up 91.47% of all sold tracks.